In [164]:
import cv2
import numpy as np

In [276]:
img = cv2.imread("mod_img.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grey = cv2.GaussianBlur(grey,(17,17), 0)
th = cv2.adaptiveThreshold(grey,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,23,2)
kernel = np.ones((1,5),np.uint8)

th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result



cv2.imwrite("inter1.jpg", result)

True

In [277]:
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()



#end
for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])


#remove smaller boxes
i = 0
others = list()
while i<len(boundRect):
    if boundRect[i][2] > 25 and boundRect[i][3] > 25:
        i += 1
    else:
        temp = boundRect[i]
        boundRect.remove(boundRect[i])
        if temp[2] >5 and temp[3] >5 :         
            others.append(temp)
        
boundRect.sort(key = lambda x:x[0])


i = 0

while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1

for i in boundRect:
    if i[2]==img3.shape[1] and i[3]==img3.shape[0]:
        boundRect.remove(i)
        break
print(boundRect)

[(39, 322, 282, 54), (43, 207, 257, 53), (61, 445, 277, 64), (393, 580, 115, 60), (398, 436, 79, 52), (410, 200, 237, 47), (418, 312, 244, 53), (518, 575, 122, 56), (534, 435, 115, 52)]


In [278]:
dropdown = set()
triangles = set()

m = np.mean(img2) +5

for small in others:
    x, y, w, h = small[0:]
    for i in range(len(boundRect)):
        x1, y1, w1, h1 = boundRect[i][0:]
        if x >= x1+w1-h1 and x+w <= x1+w1 and y >= y1 and y+h <= y1+h1:
            cv2.rectangle(img2, (x1,y1), (x1+w1, y1+h1), (255,0,0), 2)
            for j in range(x-5, x+w+5):
                for i in range(y-5, y+h+5):
                    img2[i,j] = m
            for j in range(x1-5, x1+w1+5):
                for i in range(y1-5, y1+h1+5):
                    img2[i,j] = m
            dropdown.add((x1,y1,w1,h1))
            triangles.add((x,y,w,h))
            break

for j in boundRect:
    if j in dropdown:
        boundRect.remove(j)

#sorting the rectangle as per their height and width        
boundRect.sort(key = lambda x:x[2])


#identifying date fields that are indented six boxes with a bigger box

date_fields={}

for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    date_fields[(x1,y1,w1,h1)]=[]
        
for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    for big_box in date_fields:
            x2,y2,w2,h2=big_box[0:]
            if y1>y2 and x1>x2 and y1+h1 < y2+h2 and x1+w1 < x2+w2:
                date_fields[big_box].append((x1,y1,w1,h1))
                break
        

inner_fields=set()
Date = set()

#marking date_fields and the individual boxes

for c in date_fields:
    if(len(date_fields[c]) >= 3):
        boundRect.remove(c)
        Date.add(c)
        for i in range(len(date_fields[c])):
            boundRect.remove(date_fields[c][i])
            inner_fields.add(date_fields[c][i])
        x1, y1, w1, h1 = c[:]
        for j in range(x1-5, x1+w1+5):
                for i in range(y1-5, y1+h1+5):
                    img2[i,j] = m
        

for c in inner_fields:
    x1, y1, w1, h1 = c[:]
    for j in range(x1-5, x1+w1+5):
                for i in range(y1-5, y1+h1+5):
                    img2[i,j] = m
                
TextBoxes = set()  
for c in boundRect:
        x1, y1, w1, h1 = c[:]
        TextBoxes.add(c)
        for j in range(x1-5, x1+w1+5):
            for i in range(y1-5, y1+h1+5):
                    img2[i,j] = m
    
print("DATE-FIELDS: ",Date)
print("TEXT-BOXES: ",TextBoxes)
print("INNER BOXES: ",inner_fields)
print("DROP-DOWN MENUS: ",dropdown)
print("TRIANLGLES: ",triangles)
cv2.imwrite("inter4.jpg", img2)

DATE-FIELDS:  set()
TEXT-BOXES:  {(410, 200, 237, 47), (398, 436, 79, 52), (393, 580, 115, 60), (418, 312, 244, 53), (534, 435, 115, 52), (43, 207, 257, 53)}
INNER BOXES:  set()
DROP-DOWN MENUS:  {(518, 575, 122, 56), (39, 322, 282, 54), (61, 445, 277, 64)}
TRIANLGLES:  {(292, 468, 21, 19), (271, 339, 27, 17), (595, 594, 33, 21)}


True

In [279]:
img3 = img2
Sectionlines = set()
print(img3.shape)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY) #put input image here
img3 = 255-img3
print(img3.shape)
_, img3 = cv2.threshold(img3,120,255,cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
img3 = cv2.dilate(img3, kernel, iterations = 5)
contours, h = cv2.findContours(img3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    if w>10 and h>2:
        cv2.rectangle(img2,(x,y),(x+w, y+h), (125,125,0), 2)
        Sectionlines.add((x,y,w,h))
print("SECTION-LINES: ",Sectionlines)
cv2.imshow("Image", img2)
cv2.waitKey(0)

(660, 725, 3)
(660, 725)
{(17, 90, 417, 13)}


-1